# Video Processing

In [2]:
import pandas as pd
import cv2

import csv file

In [ ]:
csv_file_path = "/SSID_IVR_Study/data/gaze_videos/SSID AV1/001_P48.csv"  
Points = pd.read_csv(csv_file_path, skiprows=lambda x: x < 26)

find the 'StartMedia' timestamp

In [ ]:
row = Points[Points['SlideEvent'] == 'StartMedia'] 
timestamp_diff = row['Timestamp'].values[0] 

## Overlaying

In [ ]:
# read the video
input_video_path = "input.mp4"
output_overlay_path = "output_overlay.mp4"

video = cv2.VideoCapture(input_video_path)
fps = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# VideoWriter mp4-mp4v format
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_overlay = cv2.VideoWriter(output_overlay_path, fourcc, fps, (width, height))

# clean the NaN in columns
Points = Points.dropna(subset=['Gaze X', 'Gaze Y'])

# - timestamp_diff
Points['Timestamp'] = Points['Timestamp'] - timestamp_diff

# loop video frames
current_point_index = 0
for frame_index in range(total_frames):
    ret, frame = video.read()
    if not ret:
        break
    
    # time of the frame in ms
    current_time = frame_index / fps * 1000 
    
    # check if the time has reached the next timestamp
    while (current_point_index < len(Points) - 1 and
           Points['Timestamp'].iloc[current_point_index + 1] <= current_time):
        current_point_index += 1 #index=index+1

    # get the coordinates corresponding to the current timestamp
    if current_point_index < len(Points):
        row = Points.iloc[current_point_index]
        x, y = int(row['Gaze X']), int(row['Gaze Y'])
        # draw a yellow circle
        cv2.circle(frame, (x, y), 50, (0, 250, 250), -1) 

    out_overlay.write(frame) 

video.release()
out_overlay.release()
cv2.destroyAllWindows()

## Chopping

In [ ]:
# read the video
input_path = output_overlay_path
output_video_path = "output_chopped.mp4"

video = cv2.VideoCapture(input_path)
fps = video.get(cv2.CAP_PROP_FPS)
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))


# VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# - timestamp difference 
Points['Timestamp'] = Points['Timestamp'] - timestamp_diff

# process frames
current_point_index = 0
skip_frames = False

for frame_index in range(total_frames):
    ret, frame = video.read()
    if not ret:
        break
    
    current_time = frame_index / fps * 1000
    
    # update current_point_index based on time 
    while current_point_index < len(Points) - 1 and Points['Timestamp'].iloc[current_point_index + 1] <= current_time:
        current_point_index += 1
        
        # skip frames
        if pd.isna(Points['Respondent Annotations active'].iloc[current_point_index]) or Points['Respondent Annotations active'].iloc[current_point_index] == '':
            skip_frames = True
        else:
            skip_frames = False

    # write frame if not skipping
    if not skip_frames:
        out.write(frame)

# release resources
video.release()
out.release()
cv2.destroyAllWindows()